In [74]:
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms,utils
import pickle
from skimage.filters.rank import entropy
from skimage.morphology import disk
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
from IPython import display
from sklearn.neural_network import MLPClassifier
from sklearn import metrics

In [72]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    #mean = np.array([0.485, 0.456, 0.406])
    #std = np.array([0.229, 0.224, 0.225])
    #inp = std * inp + mean
    #inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated
def visualize_chunks(X):
    plt.rcParams['figure.figsize'] = (20,120)
    out = torchvision.utils.make_grid( torch.from_numpy(X.transpose(0,3,1,2)),nrow = 20 )
    imshow(out)
class transfer_feature_extractor():
    def __init__(self,X,y,group,root,file, load = True):
        
        if os.path.exists(root+"/"+file+"_trans_feat.npz") and load:
            
            data = np.load(root+"/"+file+"_trans_feat.npz")
            self.VggNet_feat = data["VggNet"]
            self.ResNet_feat = data["ResNet"]
            self.y = data["y"]
            self.group = data["group"]
        else:
            self.X,self.y,self.group = self.separate_variance_filter(X,y,group)
            self.X = self.Normalize_split(self.X)
            self.VggNet_feat = self.VGG_feat(self.X)
            self.ResNet_feat = self.ResNet_feat(self.X)
            np.savez( root+"/"+file+"_trans_feat.npz", VggNet = self.VggNet_feat, ResNet = self.ResNet_feat, y = self.y, group = self.group  )
        
        if os.path.exists(root+"/"+file+"_Scat_feat.npz") and load:
            data_s = np.load(root+"/"+file+"_Scat_feat.npz")
            self.ScatX = data_s["Scat"]
        else:
            if os.path.exists(root+"/Scatfeatures/"+file+".npy"):
                self.filt_idx = self.separate_variance_filter_idx(X,y,group)
                self.ScatX = np.load(root+"/Scatfeatures/"+file+".npy")[self.filt_idx]
                np.savez( root+"/"+file+"_Scat_feat.npz", Scat = self.ScatX )
            else:
                print("No scat feature found")
        
    def Normalize_res(self, X):
        X_norm = X -  np.average(X,axis = (1,2))[:,np.newaxis,np.newaxis,:]
        stds = np.std(X_norm,axis = (1,2)) / np.array( [0.229, 0.224, 0.225] )
        # / stds[:,np.newaxis,np.newaxis,:]  - means_fix[np.newaxis,np.newaxis,np.newaxis,:]
        X_norm = np.minimum( np.maximum( X_norm/stds[:,np.newaxis,np.newaxis,:]+np.array( [0.485, 0.456, 0.406] ), 0 ),1 )
        return  X_norm
    def Normalize_split(self, X):
        num =  X.shape[0]//100
        if num < 1:
            num = 1
        X_split = np.array_split(np.arange(len(X)),num)
        for i,x_t in enumerate(X_split):
            xs = X[x_t]
            print("Normalize "+str(i)+"/"+str(num))
            if i == 0:
                feat = self.Normalize_res(xs)
            else:
                feat = np.concatenate( ( feat, self.Normalize_res(xs) ), axis=0 )
        return feat  
    def ResNet_feat(self,X):
        
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model_conv = nn.Sequential(*list(torchvision.models.resnet18(pretrained=True).children())[:-1]).double()
        for param in model_conv.parameters():
            param.requires_grad = False
        model_conv = model_conv.to(device)
        model_conv.eval()


        num = X.shape[0]//100
        if num < 1:
            num = 1
        X_split = np.array_split(np.arange(len(X)),num)

        for i,x_t in enumerate(X_split):
            xs = torch.from_numpy(X[x_t].transpose(0,3,1,2)).type(torch.DoubleTensor)
            print("ResNet "+str(i)+"/"+str(num))
            if i == 0:

                feat = np.squeeze(model_conv(xs).numpy())
            else:
                feat = np.concatenate( ( feat, np.squeeze(model_conv(xs).numpy()) ), axis=0 )
        return feat    
    def VGG_feat(self,X):
        
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        
        model_conv = torchvision.models.vgg16(pretrained=True).double()
        
        model_conv.classifier = nn.Sequential(*list(model_conv.classifier.children())[:-3])
        for param in model_conv.parameters():
            param.requires_grad = False
        model_conv = model_conv.to(device)
        model_conv.eval()
        
        num = X.shape[0]//100
        if num < 1:
            num = 1
        X_split = np.array_split(np.arange(len(X)),num)

        for i,x_t in enumerate(X_split):
            xs = torch.from_numpy(X[x_t].transpose(0,3,1,2)).type(torch.DoubleTensor)
            print("VGGNet "+str(i)+"/"+str(num))
            if i == 0:

                feat = np.squeeze(model_conv(xs).numpy())
            else:
                feat = np.concatenate( ( feat, np.squeeze(model_conv(xs).numpy()) ), axis=0 )
        return feat 
 
    def variance_filter(self,X,y,group,gp):
        stds = np.std( X[:,:,:,0]*0.299 + X[:,:,:,1]*0.587 + X[:,:,:,2]*0.114, axis = (1,2) )
        #print(np.percentile(stds, 50))
        filt = stds > np.percentile(stds, 50)
        plt.rcParams['figure.figsize'] = (10,3)
        plt.hist(stds, bins=50)  # arguments are passed to np.histogram
        plt.title(str( gp ) +" Histogram")
        plt.show()
        return X[filt], y[filt], group[filt]
    def separate_variance_filter(self,X,y,group):

        i = 0
        for gp in np.unique(group):
            idx = group == gp
            X_tmp,y_tmp,group_tmp = self.variance_filter(X[idx],y[idx],group[idx],gp)
            if i == 0:
                X_new = X_tmp
                y_new = y_tmp
                group_new = group_tmp    
            else:
                X_new = np.concatenate( (X_new,X_tmp),axis = 0 )
                y_new = np.concatenate( (y_new,y_tmp),axis = 0 )
                group_new = np.concatenate( (group_new,group_tmp),axis = 0 )
            i += 1
        return np.array(X_new), np.array(y_new), np.array(group_new)
    def variance_filter_idx(self,X,gp,idx):
        stds = np.std( X[:,:,:,0]*0.299 + X[:,:,:,1]*0.587 + X[:,:,:,2]*0.114, axis = (1,2) )
        #print(np.percentile(stds, 50))
        filt = stds > np.percentile(stds, 50)
        plt.rcParams['figure.figsize'] = (10,3)
        plt.hist(stds, bins=50)  # arguments are passed to np.histogram
        plt.title(str( gp ) +" Histogram")
        plt.show()
        return idx[filt]
    def separate_variance_filter_idx(self,X,y,group):
        X_idx = np.arange(len(X))
        i = 0
        for gp in np.unique(group):
            idx = group == gp
            X_tmp = self.variance_filter_idx(X[idx],gp,X_idx[idx])
            if i == 0:
                X_new = X_tmp
            else:
                X_new = np.concatenate( (X_new,X_tmp),axis = 0 )
            i += 1
        return np.array(X_new)
        
        


    
    
def CrossValidation(X,y,groups,scorces,feat ,typ,scl):

    SVM = svm.LinearSVC()
    RF = RandomForestClassifier(max_depth=None, n_estimators=200)
    LR = LogisticRegression()


    est = {'SVM': SVM,  'LR': LR, 'NN': MLPClassifier(alpha=1), }




    for estimator in est.keys():
        '''
        for fold in [5, 10, 21]:
            # GroupKFold
            clf = est[estimator]

            scores = cross_val_score(clf, X , y, cv=GroupKFold(n_splits=fold),groups =  groups)
            record = {"Estimator":estimator, "Fold":fold,"CV method":'GroupKFold',
                      "Acc":"%.3f"%np.average(scores),"Std" : "%.3f"%(2*np.std( scores )/np.sqrt(fold)),
                         "type": typ, "scale": scl, "feature": feat
                     }
            scorces.append(record)

        # Image Leave one out:
        '''
        res = []
        gps = []
        im_label = []
        prediction_score = []
        for gp in np.unique( groups ):
            val_idx = groups == gp
            train_idx = groups != gp
            #print(train_idx)
            clf.fit(X[train_idx],y[train_idx])
            res.append(clf.score(X[val_idx],y[val_idx]))
            gps.append(gp)
            im_label.append(y[val_idx][0])
            if im_label == 0:
                prediction_score.append(res[-1])
            else:
                prediction_score.append(1-res[-1])
                
        fpr, tpr, thresholds = metrics.roc_curve(im_label, prediction_score, pos_label=1)
        AUC = metrics.auc(fpr, tpr)
        
        sortidx =  np.argsort(res)
        min2 = str( gps[sortidx[0]] ) + ", " + str( gps[sortidx[1]] )
        max2 = str( gps[sortidx[-1]] ) + ", " + str( gps[sortidx[-2]] )
        record = {"Estimator":estimator,"CV method":'ImageLOO',"Acc":"%.3f"%np.average(res),"Std" : "%.3f"%(2*np.std( res )/np.sqrt(fold)),
                  "Max_2":max2, "Min_2":min2,"AUC":"%.3f"%AUC,
                 "type": typ, "scale": scl, "feature": feat
                 }
        scorces.append(record)
        df = pd.DataFrame(scorces)
        display.clear_output(wait=True)
        print(df)
        
    return scorces


In [76]:
img_root='/Users/scraed/Downloads/data+label/TrainData'
scores = []
for scale in [1,0.5,0.25,0.125]:
    for typ in ["sequential", "entropy_random"]:
        file_name = str(scale)+typ+".npz"
        data = np.load(img_root + "/" + file_name)
        X = data["X"]
        y = data["y"]
        group = data["group"]
        tr_feat = transfer_feature_extractor(X,y,group,img_root,file_name,load = True)
        scores = CrossValidation(tr_feat.ResNet_feat, tr_feat.y, tr_feat.group, scores,"ResNet" ,typ,scale)
        scores = CrossValidation(tr_feat.VggNet_feat, tr_feat.y, tr_feat.group, scores,"Vgg" ,typ,scale)
        scores = CrossValidation(tr_feat.ScatX, tr_feat.y, tr_feat.group, scores,"Scat" ,typ,scale)
        
    
        
        

      AUC    Acc CV method Estimator   Max_2   Min_2    Std feature  scale  \
0   0.634  0.625  ImageLOO       SVM   24, 2   14, 5  0.136  ResNet  1.000   
1   0.634  0.625  ImageLOO        LR   24, 2   14, 5  0.136  ResNet  1.000   
2   0.634  0.625  ImageLOO        NN   24, 2   14, 5  0.136  ResNet  1.000   
3   0.657  0.604  ImageLOO       SVM  28, 21  14, 27  0.146     Vgg  1.000   
4   0.657  0.604  ImageLOO        LR  28, 21  14, 27  0.146     Vgg  1.000   
5   0.657  0.604  ImageLOO        NN  28, 21  14, 27  0.146     Vgg  1.000   
6   0.505  0.637  ImageLOO       SVM   14, 2   5, 27  0.149    Scat  1.000   
7   0.505  0.637  ImageLOO        LR   14, 2   5, 27  0.149    Scat  1.000   
8   0.505  0.637  ImageLOO        NN   14, 2   5, 27  0.149    Scat  1.000   
9   0.657  0.591  ImageLOO       SVM   21, 3   27, 4  0.144  ResNet  1.000   
10  0.657  0.591  ImageLOO        LR   21, 3   27, 4  0.144  ResNet  1.000   
11  0.657  0.591  ImageLOO        NN   21, 3   27, 4  0.144  Res

In [68]:
df = pd.DataFrame(scores).sort_values(by=['Acc'],ascending=False)
df

,AUC,Acc,CV method,Estimator,Fold,Max_2,Min_2,Std,feature,scale,type
220,NaN,0.776,GroupKFold,LR,5.0,NaN,NaN,0.180,ResNet,0.125,entropy_random
12,NaN,0.759,GroupKFold,LR,5.0,NaN,NaN,0.059,Vgg,0.000,sequential
8,NaN,0.754,GroupKFold,SVM,5.0,NaN,NaN,0.051,Vgg,0.000,sequential
21,NaN,0.753,GroupKFold,LR,10.0,NaN,NaN,0.152,Scat,0.000,sequential
216,NaN,0.752,GroupKFold,SVM,5.0,NaN,NaN,0.170,ResNet,0.125,entropy_random
188,NaN,0.748,GroupKFold,LR,5.0,NaN,NaN,0.066,Scat,0.250,entropy_random
186,NaN,0.748,GroupKFold,SVM,21.0,NaN,NaN,0.161,Scat,0.250,entropy_random
168,NaN,0.743,GroupKFold,SVM,5.0,NaN,NaN,0.104,ResNet,0.250,entropy_random
172,NaN,0.742,GroupKFold,LR,5.0,NaN,NaN,0.109,ResNet,0.250,entropy_random
223,0.727,0.740,ImageLOO,LR,NaN,"2, 12","28, 11",0.156,ResNet,0.125,entropy_random


In [70]:
df.to_csv(img_root+"CV.csv", encoding='utf-8')